<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

Схема данных для работы над проектом приведена [здесь](https://github.com/esmelnikova/education_dst_128/tree/main/Project_2/description/Схема_данных.md)

In [588]:
import pandas as pd
import psycopg2

In [589]:
# параметры подключения к БД


In [590]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

In [591]:
# отключить предупреждения в ноутбуке о несовместимости версий библиотек
import warnings
warnings.filterwarnings('ignore')

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [592]:
# текст запроса
query_3_1 = f'''select count(*) from public.vacancies'''

In [593]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
display(df)

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [594]:
query_3_2 = f'''select count(*) from public.employers'''

In [595]:
df = pd.read_sql_query(query_3_2, connection)
display(df)

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [596]:
query_3_3 = f'''select count(*) from public.areas'''

In [597]:
df = pd.read_sql_query(query_3_3, connection)
display(df)

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [598]:
query_3_4 = f'''select count(*) from public.industries'''

In [599]:
df = pd.read_sql_query(query_3_4, connection)
display(df)

,count
0,294


***

**Выводы по предварительному анализу данных**

Схема данных содержит 5 основных таблиц для анализа вакансий:
* public.vacancies (вакансии)
* public.employers (работодатели)
* public.areas (регионы)
* public.industries (сферы деятельности предприятий)
* public.employers_industries (для связи "много-ко-многим" между таблицами employers и industries)
    
Таблицы не пустые, данные во вакансиям присутсвуют в достаточном количестве (около 50 тыс. вакансий у 23 тыс. работодателей), 
возможен переход к более детальному анализу данных.

Справочники регионов и сфер деятельности содержат множество неповторяющихся значений (соответсвенно 1392 и 294), что делает эти признаки слабоинформативным для анализа. Рекомендуется в дальнейшем преобразование данных с объединением признаков в группы.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [600]:
# кол-во вакансий в регионах
query_4_1 = f'''
    SELECT a.name as area, count(v.id) as cnt
    FROM public.areas a 
    LEFT JOIN public.vacancies v--внешнее соединение для регионов без вакансий
    ON v.area_id=a.id
    GROUP BY a.id
    ORDER BY cnt DESC
'''
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
display(df)

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
1357,Балакирево,0
1358,Мексика,0
1359,Пугачев,0
1360,Долгоруково,0


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [601]:
query_4_2 = f'''
    SELECT count(*) 
    FROM public.vacancies
    WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
'''

In [602]:
df = pd.read_sql_query(query_4_2, connection)
display(df)

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [603]:
# исследование данных
# запрос заполненных зарплатных вилок

query_4_3 = f'''
    SELECT salary_from, salary_to
    FROM public.vacancies
    WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
'''

#результат запроса
df = pd.read_sql_query(query_4_3, connection)
display(df)

,salary_from,salary_to
0,64000.0,NaN
1,15000.0,NaN
2,40000.0,60000.0
3,50000.0,NaN
4,150000.0,NaN
...,...,...
24068,50000.0,NaN
24069,102500.0,NaN
24070,40000.0,78000.0
24071,60000.0,NaN


In [604]:
# запрос средних значений зарплатных вилок
query_4_3 = f'''
    SELECT round(avg(salary_from)) as avg_salary_from,
	       round(avg(salary_to)) as avg_salary_to
    FROM public.vacancies
'''

#результат запроса
df = pd.read_sql_query(query_4_3, connection)
display(df)

,avg_salary_from,avg_salary_to
0,71065.0,110537.0


In [605]:
# доп. запрос
# минимальные и максимальные значения для границ зарплатной вилки
query_4_3 = f'''
    SELECT  min(salary_from) as min_salary_from, max(salary_from) as max_salary_from,
	        min(salary_to) as min_salary_to, max(salary_to) as max_salary_to
    FROM public.vacancies
'''

# реузльтат запроса
df = pd.read_sql_query(query_4_3, connection)
display(df)

,min_salary_from,max_salary_from,min_salary_to,max_salary_to
0,30,1000000,60,1000000


In [606]:
# доп. запрос
# минимальные и максимальные значения для границ зарплатной вилки в зависимости от региона
query_4_3 = f'''
    SELECT  a. name,
	        min(salary_from) as min_salary_from, max(salary_from) as max_salary_from,
	        min(salary_to) as min_salary_to, max(salary_to) as max_salary_to
    FROM public.vacancies v
	JOIN public.areas a ON v.area_id=a.id
	GROUP BY a.id
	ORDER BY min_salary_from, max_salary_from DESC, min_salary_to, max_salary_to DESC
'''

# реузльтат запроса
df = pd.read_sql_query(query_4_3, connection)
df

,name,min_salary_from,max_salary_from,min_salary_to,max_salary_to
0,Сочи,30.0,500000.0,25000.0,1000000.0
1,Челябинск,30.0,300000.0,6000.0,350000.0
2,Воронеж,35.0,270000.0,25000.0,500000.0
3,Смоленск,60.0,180000.0,60.0,300000.0
4,Нур-Султан,65.0,400000.0,12205.0,526036.0
...,...,...,...,...,...
764,Риддер (ВКО),NaN,NaN,NaN,NaN
765,Финляндия,NaN,NaN,NaN,NaN
766,Мегион,NaN,NaN,NaN,NaN
767,Индонезия,NaN,NaN,NaN,NaN


In [607]:
# доп. запрос
# минимальные и максимальные значения для границ зарплатной вилки в зависимости от сферы деятельности
query_4_3 = f'''
    SELECT  ind.name,
	        min(salary_from) as min_salary_from, max(salary_from) as max_salary_from,
	        min(salary_to) as min_salary_to, max(salary_to) as max_salary_to
    FROM public.vacancies v
	JOIN public.employers emp ON v.employer_id=emp.id
	JOIN public.employers_industries emp_ind ON emp.id=emp_ind.employer_id
	JOIN public.industries ind ON emp_ind.industry_id=ind.id
	GROUP BY ind.id
	ORDER BY min_salary_from, max_salary_from DESC, min_salary_to, max_salary_to DESC
'''

df = pd.read_sql_query(query_4_3, connection)
df

,name,min_salary_from,max_salary_from,min_salary_to,max_salary_to
0,Строительство жилищное,30.0,500000.0,21206.0,200000.0
1,Агентские услуги в недвижимости,30.0,500000.0,25000.0,1000000.0
2,"Безопасность, охранная деятельность",30.0,160000.0,17064.0,140000.0
3,Государственные организации,35.0,250000.0,12205.0,300000.0
4,Разработка программного обеспечения,42.0,500000.0,850.0,1000000.0
...,...,...,...,...,...
288,Благотворительная организация,50000.0,50000.0,60000.0,60000.0
289,"Рыболовство, рыбоводство",60000.0,60000.0,NaN,NaN
290,Факторинговые компании,100000.0,100000.0,NaN,NaN
291,Автошкола,NaN,NaN,NaN,NaN


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [608]:
query_4_4 = f'''
    SELECT format('%s - %s',schedule,employment) as type, count(id)
    FROM public.vacancies
    GROUP BY schedule, employment
    ORDER BY count DESC
'''

In [609]:
df = pd.read_sql_query(query_4_4, connection)
display(df)

,type,count
0,Полный день - Полная занятость,35367
1,Удаленная работа - Полная занятость,7802
2,Гибкий график - Полная занятость,1593
3,Удаленная работа - Частичная занятость,1312
4,Сменный график - Полная занятость,940
5,Полный день - Стажировка,569
6,Вахтовый метод - Полная занятость,367
7,Полный день - Частичная занятость,347
8,Гибкий график - Частичная занятость,312
9,Полный день - Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [610]:
query_4_5 = f'''
    SELECT experience, count(id)
    FROM public.vacancies
    GROUP BY experience
    ORDER BY count
'''

In [611]:
df = pd.read_sql_query(query_4_5, connection)
display(df)

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

**Выводы по детальному анализу вакансий**

* Наибольшее количество вакансий размещены в крупных городах-миллионниках (Москва, Санкт-Петербург, Новосибирск, Екатеринбург и т.д.). Москва - абсолютный лидер в этом рейтинге, более 5 тыс. вакансий. За ним следует Санкт-Петербург, где около 2800 вакансий.

* Рассматриваются вакансии не только на территории РФ, но и в зарубежных странах (Республика Беларусь, Казахстан, Узбекистан, Литва, Сербия, Германия и др.)

* Данные о границах зарплатой вилки заполнены только в половине вакансий (примерно 24 тыс. из 49 тыс. вакансий), этот факт необходимо учесть при разработке алгоритмов рекомендаций вакансий (Например, предсказывать эти значения из аналогичных вакансий, заполнять модальным значением или задать низкий весовой коэфиициент для этого признака при отборе подходящих вакансий и т.д.)

* Разброс значений для границ зарплатной вилки слишком велик (нижняя граница: 30-1000000 руб., верхняя: 60-1000000 руб.), что делает расчет среднего значения для этих параметров не информативным. Кроме того, разброс уровня ЗП может существенно различаться в зависимости от региона и сферы деятельности предприятия. Рекомендуется дополнительно исследовать эти параметры, например построить распределение, вычислить модальное значение, проследить корреляцию с выше указанными признаками.

* Присутствуют потенциально аномальные значения в зарплатной вилке (слишком низкие значения: 30, 70 руб. и т.д.), возможно данные ошибочно загружены не в рублях или не учтена кратность, например тыс.руб.

* В большинстве вакансий (ок. 70%) требуются сотрудники на полный рабочий день с полной занятостью, 15% вакансий рассчитаны на удаленную работу с полной занятостью.

* В большинсте вакансий (50%) требуются соискатели с опытом работы от 1 года до 3 лет, в 30% вакансий необходимый опыт составляет от 3 до 6 лет. Но присутствует достаточно большое кол-во вакансий (15%), где готовы взять соискателя без опыта работы.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [612]:
query_5_1 = f'''
    SELECT e.name, count(v.id)
    FROM public.vacancies v
    JOIN public.employers e
    ON v.employer_id=e.id
    GROUP BY e.id
    ORDER BY count DESC
    LIMIT 5
'''

In [613]:
df = pd.read_sql_query(query_5_1, connection)
df

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [614]:
query_5_2 = f'''
    SELECT a.name, count(distinct e.id) emp_count, count(distinct v.id) vac_count --считаем кол-во уникальных(!) компаний в регионе
    FROM public.areas a
    LEFT JOIN public.employers e ON a.id=e.area --может не быть работодателей
    LEFT JOIN public.vacancies v ON v.area_id=a.id --может не быть вакансий
    GROUP BY a.id
    ORDER BY vac_count ASC, emp_count DESC
'''

In [615]:
df = pd.read_sql_query(query_5_2, connection)
df

,name,emp_count,vac_count
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
1357,Алматы,721,1892
1358,Новосибирск,573,2006
1359,Минск,1115,2112
1360,Санкт-Петербург,2217,2851


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [616]:
query_5_3 = f'''
    SELECT e.name, count(distinct v.area_id) as cnt_areas --считаем кол-во уникальных(!) регионов для компании
    FROM public.employers e
    LEFT JOIN public.vacancies v --может не быть вакансий у работодателя
    ON v.employer_id=e.id
    GROUP BY e.id
    ORDER BY cnt_areas desc
'''

In [617]:
df = pd.read_sql_query(query_5_3, connection)
display(df)

,name,cnt_areas
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23496,Но Смокинг,0
23497,МАСТЕРСКАЯ CREED,0
23498,Dzing Finance,0
23499,КОРС,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [618]:
query_5_4 = f'''
    SELECT count(e.id)
    FROM public.employers e --сфера может быть не указана, используем внешнее соединение
    LEFT JOIN public.employers_industries ei ON e.id=ei.employer_id 
    LEFT JOIN public.industries i ON i.id=ei.industry_id
    WHERE i.id is NULL
'''

# альтернативный вариант запроса: 
# проверить связь работодателя и сферы деятельности только через таблицу employers_industries
# если там нет записей о работодателе, то значит сфера не указана
query_5_4_alt = f'''
    SELECT count(e.id)
    FROM public.employers e --сфера может быть не указана, используем внешнее соединение
    LEFT JOIN public.employers_industries ei ON e.id=ei.employer_id 
    WHERE ei.employer_id is NULL
'''

In [619]:
df = pd.read_sql_query(query_5_4_alt, connection)
display(df)

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [620]:
query_5_5 = f'''
    SELECT e.name, count(i.id)
    FROM public.employers e 
    JOIN public.employers_industries ei ON e.id=ei.employer_id 
    JOIN public.industries i ON i.id=ei.industry_id
    GROUP BY e.id
    HAVING count(i.id)=4
    ORDER BY name
'''
# альтернативный вариант запроса: 
# посчитать кол-во сфер деятельности только через таблицу employers_industries
# пара employer_id-industry_id является уникальной (PK для employers_industries), дубликатов при подсчете не будет
query_5_5_alt = f'''
    SELECT e.name, count(ei.industry_id)
    FROM public.employers e 
    JOIN public.employers_industries ei ON e.id=ei.employer_id 
    GROUP BY e.id
    HAVING count(ei.industry_id)=4
    ORDER BY name
'''

In [621]:
df = pd.read_sql_query(query_5_5_alt, connection)
df

,name,count
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4
...,...,...
1133,ЮРИОН,4
1134,ЮТИП Технологии,4
1135,ЯКласс,4
1136,ЯрНео,4


In [622]:
# доп. запрос
# компании, у которых указано несколько сфер деятельности
query_5_5_1 = f'''
    SELECT e.name, count(i.id)
    FROM public.employers e 
    JOIN public.employers_industries ei ON e.id=ei.employer_id 
    JOIN public.industries i ON i.id=ei.industry_id
    GROUP BY e.id
    HAVING count(i.id)>1
    ORDER BY name
'''

df = pd.read_sql_query(query_5_5_1, connection)
df

,name,count
0,101 Интернет,4
1,144 бронетанковый ремонтный завод,2
2,169,5
3,16-я городская детская поликлиника г.Минска,2
4,179 СРЗ,2
...,...,...
8625,Ясли-сад №374 г. Минска,2
8626,ЯСП,2
8627,Ястро-Переработка,3
8628,ЯТСС,3


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [623]:
query_5_6 = f'''
    SELECT count(*)
    FROM public.employers e 
    JOIN public.employers_industries ei ON e.id=ei.employer_id 
    JOIN public.industries i ON i.id=ei.industry_id
    WHERE i.name like 'Разработка программного обеспечения'
'''

In [624]:
df = pd.read_sql_query(query_5_6, connection)
display(df)

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [625]:
# получение списка городов-милионников

# найти таблицы на странице по ключевому слову
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' 
city_tb = pd.read_html(url, match='город')

#получить из таблицы список названий городов
city_ls = list(city_tb[0]['Город']) if len(city_tb)>0 else []
print(city_ls)

['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж[a]', 'Пермь', 'Волгоград']


In [626]:
# Исправить ошибку в названии города
city_ls[13]='Воронеж'
print(city_ls)

['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград']


In [627]:
# запрос кол-ва вакансий в городах-миллионниках

emp_name='Яндекс'
cities = tuple(city_ls)

query_5_7 = f'''
WITH mc_vac AS (
	SELECT mc.name, count(vac.id) as cnt 
 	FROM
		(select id, name from public.areas where name in {cities}
		) as mc
 	JOIN public.vacancies vac ON vac.area_id=mc.id
 	JOIN public.employers emp ON emp.id=vac.employer_id
 	WHERE emp.name like '{emp_name}'
 	GROUP BY mc.name
 ) --CTE выполняется единожды
 SELECT * FROM mc_vac
 UNION ALL
 SELECT 'Total', sum(cnt) from mc_vac
 ORDER BY cnt
'''

In [628]:
df = pd.read_sql_query(query_5_7, connection)
df

,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


***

**Выводы по анализу работодателей**

* Всего в БД хранятся данные о 23501 работодателе.

* Наибольшее количество вакансий ожидаемо зарегистрированы от имени крупных компаний таких как: Яндекс, Ростелеком, Тинькофф, Сбер и т.д. Но абсолютным лидером является Яндекс, они предлагают в 4 раза больше вакансий, чем другие крупные компании.

* Наибольшее количество работодателей и вакансий размещены в крупных городах. Например в Москве находятся почти 25% из всех работодалей, сохраненных в БД, в Санкт-Петербурге 9% и далее этот процент уменьшается для остальных регионов.

* Среди регионов с доступными вакансиями прослеживается линейная зависимость кол-ва работодателей и числа предложений. Среди регионов, в которых нет вакансий, по количеству работодателей лидируют "Россия", "Казахстан".

* Лидерами по количеству регионов, в которых компания разместила вакансии, являются Яндекс и Ростелеком.

* У трети работодателей указано более 1 сферы деятельности.

* Около 15% работодателей ищут соискателей в области разработки ПО.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [629]:
# исследование данных
# список вакансий с упоминанием 'data'/'данн' 

query_6_1 = f'''
	select *
	from public.vacancies
	where lower(name) like '%data%' OR lower(name) like '%данн%'
'''

# результат запроса
df = pd.read_sql_query(query_6_1, connection)
df

,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,66075509,Аналитик хранилищ данных и BI-систем/Аналитик ...,SQL\tREST API\tBI-систем/ EAM(ТОиР)\tПроектная...,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,43,8645304
1,55462336,Data-analysts (удаленно),SQL\tPython\tPostgreSQL\tDocker\tTableau,Удаленная работа,От 1 года до 3 лет,Полная занятость,NaN,NaN,98,150082
2,55348138,Стажер Data Engineer/Data Аналитик в команду Б...,Python\tSQL\tMS PowerPoint,Полный день,Нет опыта,Стажировка,NaN,NaN,1,3529
3,54629664,Старший специалист хранилищ данных,SQL\tORACLE\tБазы данных\tETL\tБаза данных: Or...,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,88,7944
4,55348337,Администратор баз данных и сетей,None,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,246,194646
...,...,...,...,...,...,...,...,...,...,...
1766,66079596,Программист Delphi / разработчик баз данных,None,Полный день,От 1 года до 3 лет,Полная занятость,80000.0,NaN,26,81154
1767,55219239,Разработчик баз данных (Big Data),SQL\tАнглийский язык\tORACLE\tMS SQL\tБазы дан...,Удаленная работа,От 1 года до 3 лет,Полная занятость,NaN,NaN,75,3776
1768,55900304,Аналитик данных,Работа с большим объемом информации\tАнализ да...,Полный день,От 1 года до 3 лет,Полная занятость,70000.0,70000.0,2,2953546
1769,66185897,Data аналитик /Аналитик BI (удаленно),Python\tTableau\tSQL\tLinux\tОбучение и развитие,Удаленная работа,От 1 года до 3 лет,Полная занятость,NaN,NaN,26,84585


In [630]:
# кол-во вакансий с упоминанием 'data'/'данн'
query_6_1 = f'''
	select count(*)
	from public.vacancies
	where lower(name) like '%data%' OR lower(name) like '%данн%'
'''

# результат запроса
df = pd.read_sql_query(query_6_1, connection)
display(df)

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [631]:
# исследование данных
# вакансии DS
query_6_2 = f'''
SELECT *
FROM public.vacancies
where (lower(name) like '%data scientist%' OR 
	   lower(name) like '%data science%' OR 
	   lower(name) like '%исследователь данных%' OR 
	   (name like '%ML%' and name NOT like '%HTML%' and NOT name like '%QML%') OR --дополнительно исключаем лишние вакансии
	   lower(name) like '%machine learning%' OR 
	   lower(name) like '%машинн%обучен%')
'''

# результат запроса
df = pd.read_sql_query(query_6_2, connection)
df

,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,50533416,Junior+ CV/ML developer,Python\tSQL\tРабота в команде\tOpenCV\tML,Гибкий график,От 1 года до 3 лет,Полная занятость,NaN,NaN,4,80190
1,54824572,Data Scientist,Oracle Pl/SQL\tБазы данных\tMachine Learning\t...,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,160,111304
2,54825318,Data Scientist (Junior+),Python\tOpenCV\tTensorflow\tComputer Vision\tН...,Полный день,От 1 года до 3 лет,Полная занятость,50000.0,70000.0,72,215942
3,66120847,AI/ML Senior Engineer,Python\tFlask\tLinux\tAWS\tGit,Гибкий график,От 3 до 6 лет,Полная занятость,NaN,NaN,2759,204511
4,55351147,Data Scientist,Python\tSQL\tNLP\tScikit-learn\tMachine Learning,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,53,3127
...,...,...,...,...,...,...,...,...,...,...
462,55306481,Data scientist / Исследователь-программист данных,Python\tАналитическое мышление\tАнализ данных\...,Полный день,От 1 года до 3 лет,Полная занятость,115000.0,172000.0,1,4846045
463,54918116,Специалист ML/DS,Python\tNumpy\tPyTorch\tPandas\tМатематический...,Полный день,От 3 до 6 лет,Полная занятость,61024.0,73228.0,160,5969955
464,55185186,Data Scientist в области рекомендательных систем,Python\tScikit-learn\tPandas\tGit\tNumpy,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,66,3529
465,66125447,Middle+/Senior Data Scientist,SCALA\tSQL\tМатематический анализ\tСтатистичес...,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,160,135317


In [632]:
# вакансии DS для начинающего специалиста
query_6_2 = f'''
SELECT *
FROM public.vacancies
where (lower(name) like '%data scientist%' OR 
	   lower(name) like '%data science%' OR 
	   lower(name) like '%исследователь данных%' OR 
	   (name like '%ML%' and name NOT like '%HTML%' and NOT name like '%QML%') OR --дополнительно исключаем лишние вакансии
	   lower(name) like '%machine learning%' OR 
	   lower(name) like '%машинн%обучен%') AND 
	   (lower(name) like '%junior%' OR lower(experience) like 'нет опыта' OR  lower(schedule) like 'стажировка')
'''
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
display(df.head(10))

,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,50533416,Junior+ CV/ML developer,Python\tSQL\tРабота в команде\tOpenCV\tML,Гибкий график,От 1 года до 3 лет,Полная занятость,NaN,NaN,4,80190
1,54825318,Data Scientist (Junior+),Python\tOpenCV\tTensorflow\tComputer Vision\tН...,Полный день,От 1 года до 3 лет,Полная занятость,50000.0,70000.0,72,215942
2,55553897,Junior Data Scientist (Стажер),Python\tSQL\tData Mining\tC++\tМатематический ...,Гибкий график,Нет опыта,Стажировка,NaN,37000.0,3,953
3,55145641,Junior Data Scientist,Python\tSQL\tMachine Learning\tData Science\tА...,Полный день,Нет опыта,Полная занятость,50000.0,NaN,2,40938
4,55529384,"Математик, Junior/Middle/Senior Data Scientist",Python\tMATLAB\tМатематическое моделирование\t...,Удаленная работа,От 1 года до 3 лет,Частичная занятость,NaN,NaN,1002,1688252
5,55343709,Data Scientist/Machine Learning Engineer (Deve...,Machine Learning\tDeep Learning\tData Science\...,Гибкий график,Нет опыта,Частичная занятость,NaN,NaN,2,675403
6,55120211,Junior Data Scientist (VK),Python\tМатематическая статистика\tАнализ данных,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,1,15478
7,54475492,Data Scientist (Junior),Python\tMachine Learning\tPandas\tSQL\tNumpy\t...,Полный день,Нет опыта,Частичная занятость,45000.0,100000.0,1,665449
8,55119989,Intern/Junior Data Scientist,Linux\tPython,Гибкий график,Нет опыта,Стажировка,NaN,NaN,2,3124
9,55120401,Junior/Middle Data Scientist,Linux\tPython,Гибкий график,От 1 года до 3 лет,Полная занятость,NaN,NaN,2,3124


In [633]:
# запрос для вывода кол-ва вакансий DS для начинающих специалистов
query_6_2 = f'''
SELECT count(*)
FROM public.vacancies
where (lower(name) like '%data scientist%' OR 
	   lower(name) like '%data science%' OR 
	   lower(name) like '%исследователь данных%' OR 
	   (name like '%ML%' and name NOT like '%HTML%' and NOT name like '%QML%') OR --дополнительно исключаем лишние вакансии
	   lower(name) like '%machine learning%' OR 
	   lower(name) like '%машинн%обучен%') AND 
	   (lower(name) like '%junior%' OR lower(experience) like 'нет опыта' OR  lower(schedule) like 'стажировка')
'''

# кол-во вакансий
df = pd.read_sql_query(query_6_2, connection)
display(df)

,count
0,51


In [634]:
# доп. исследование
# кол-во вакансий DS в зависимости от опыта работы

query_6_2 = f'''
SELECT experience, count(id)
FROM public.vacancies
where (lower(name) like '%data scientist%' OR 
	   lower(name) like '%data science%' OR 
	   lower(name) like '%исследователь данных%' OR 
	   (name like '%ML%' and name NOT like '%HTML%' and NOT name like '%QML%') OR --дополнительно исключаем лишние вакансии
	   lower(name) like '%machine learning%' OR 
	   lower(name) like '%машинн%обучен%') 
GROUP BY experience
'''

# кол-во вакансий
df = pd.read_sql_query(query_6_2, connection)
display(df)

,experience,count
0,Более 6 лет,34
1,Нет опыта,28
2,От 1 года до 3 лет,219
3,От 3 до 6 лет,186


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [635]:
# запрос для визуального исследования данных (выводим всю информацию по выбранным вакансиям)
query_6_3 = f'''
SELECT *
FROM public.vacancies
where (lower(name) like '%data scientist%' OR 
	   lower(name) like '%data science%' OR 
	   lower(name) like '%исследователь данных%' OR 
	   (name like '%ML%' and name NOT like '%HTML%' and NOT name like '%QML%') OR --дополнительно исключаем лишние вакансии
	   lower(name) like '%machine learning%' OR 
	   lower(name) like '%машинн%обучен%') AND 
	   (lower(key_skills) like '%sql%' OR lower(key_skills) like '%postgres%')
'''

# результат запроса
df = pd.read_sql_query(query_6_3, connection)
display(df.head(10))

,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,50533416,Junior+ CV/ML developer,Python\tSQL\tРабота в команде\tOpenCV\tML,Гибкий график,От 1 года до 3 лет,Полная занятость,NaN,NaN,4,80190
1,54824572,Data Scientist,Oracle Pl/SQL\tБазы данных\tMachine Learning\t...,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,160,111304
2,55351147,Data Scientist,Python\tSQL\tNLP\tScikit-learn\tMachine Learning,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,53,3127
3,55351168,Data Scientist,Python\tSQL\tNLP\tScikit-learn\tMachine Learning,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,78,3127
4,55351179,Data Scientist,Python\tSQL\tNLP\tScikit-learn\tMachine Learning,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,76,3127
5,55351195,Data Scientist,Python\tSQL\tNLP\tScikit-learn\tMachine Learning,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,2,3127
6,55553897,Junior Data Scientist (Стажер),Python\tSQL\tData Mining\tC++\tМатематический ...,Гибкий график,Нет опыта,Стажировка,NaN,37000.0,3,953
7,55299643,Риск-менеджер/Data Scientist,Python\tSQL\tSas\tMS PowerPoint\tАнализ данных,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,160,123401
8,54659098,Data Scientist / ML-разработчик,Python\tSQL\tML\tMachine Learning,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,1,2942396
9,55145641,Junior Data Scientist,Python\tSQL\tMachine Learning\tData Science\tА...,Полный день,Нет опыта,Полная занятость,50000.0,NaN,2,40938


In [636]:
# запрос кол-ва вакансий DS по выбранным условиям
query_6_3 = f'''
SELECT count(*)
FROM public.vacancies
where (lower(name) like '%data scientist%' OR 
	   lower(name) like '%data science%' OR 
	   lower(name) like '%исследователь данных%' OR 
	   (name like '%ML%' and name NOT like '%HTML%' and NOT name like '%QML%') OR --дополнительно исключаем лишние вакансии
	   lower(name) like '%machine learning%' OR 
	   lower(name) like '%машинн%обучен%') AND 
	   (lower(key_skills) like '%sql%' OR lower(key_skills) like '%postgres%')
'''

# кол-во вакансий
df = pd.read_sql_query(query_6_3, connection)
display(df)

,count
0,200


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [637]:
# запрос для визуального исследования данных (выводим всю информацию по выбранным вакансиям)
query_6_4 = f'''
SELECT *
FROM public.vacancies
where (lower(name) like '%data scientist%' OR 
	   lower(name) like '%data science%' OR 
	   lower(name) like '%исследователь данных%' OR 
	   (name like '%ML%' and name NOT like '%HTML%' and NOT name like '%QML%') OR --дополнительно исключаем лишние вакансии
	   lower(name) like '%machine learning%' OR 
	   lower(name) like '%машинн%обучен%') AND 
	   lower(key_skills) like '%python%'
'''

# результат запроса
df = pd.read_sql_query(query_6_4, connection)
display(df.head(10))

,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,50533416,Junior+ CV/ML developer,Python\tSQL\tРабота в команде\tOpenCV\tML,Гибкий график,От 1 года до 3 лет,Полная занятость,NaN,NaN,4,80190
1,54824572,Data Scientist,Oracle Pl/SQL\tБазы данных\tMachine Learning\t...,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,160,111304
2,54825318,Data Scientist (Junior+),Python\tOpenCV\tTensorflow\tComputer Vision\tН...,Полный день,От 1 года до 3 лет,Полная занятость,50000.0,70000.0,72,215942
3,66120847,AI/ML Senior Engineer,Python\tFlask\tLinux\tAWS\tGit,Гибкий график,От 3 до 6 лет,Полная занятость,NaN,NaN,2759,204511
4,55351147,Data Scientist,Python\tSQL\tNLP\tScikit-learn\tMachine Learning,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,53,3127
5,55351168,Data Scientist,Python\tSQL\tNLP\tScikit-learn\tMachine Learning,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,78,3127
6,55351179,Data Scientist,Python\tSQL\tNLP\tScikit-learn\tMachine Learning,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,76,3127
7,55351195,Data Scientist,Python\tSQL\tNLP\tScikit-learn\tMachine Learning,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,2,3127
8,66120842,AI/ML Senior Engineer,Python\tFlask\tLinux\tAWS\tGit,Гибкий график,От 3 до 6 лет,Полная занятость,NaN,NaN,2758,204511
9,55166994,ML разработчик (удалённо),Python\tПоиск информации в интернет\tC++\tМате...,Удаленная работа,От 1 года до 3 лет,Полная занятость,50000.0,100000.0,1,6154479


In [638]:
# запрос кол-ва вакансий DS по выбранным условиям
query_6_4 = f'''
SELECT count(*)
FROM public.vacancies
where (lower(name) like '%data scientist%' OR 
	   lower(name) like '%data science%' OR 
	   lower(name) like '%исследователь данных%' OR 
	   (name like '%ML%' and name NOT like '%HTML%' and NOT name like '%QML%') OR --дополнительно исключаем лишние вакансии
	   lower(name) like '%machine learning%' OR 
	   lower(name) like '%машинн%обучен%') AND 
	   lower(key_skills) like '%python%'
'''

# кол-во вакансий
df = pd.read_sql_query(query_6_4, connection)
display(df)

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [639]:
# запрос для визуального исследования данных (до группировки)
query_6_5 = f'''
SELECT 
	key_skills as orig_skills,
	REPLACE(key_skills, CHR(9), '_') as modif_skills,
	length(key_skills) - length(REPLACE(key_skills, CHR(9), '')) + 1 as skills_count --в конце строки отсутсвует табуляция! +1 к кол-ву.
FROM public.vacancies
where (lower(name) like '%data scientist%' OR 
	   lower(name) like '%data science%' OR 
	   lower(name) like '%исследователь данных%' OR 
	   (name like '%ML%' and name NOT like '%HTML%' and NOT name like '%QML%') OR --дополнительно исключаем лишние вакансии
	   lower(name) like '%machine learning%' OR 
	   lower(name) like '%машинн%обучен%') AND
	   key_skills IS NOT NULL
'''

# результаты запроса
df = pd.read_sql_query(query_6_5, connection)
display(df.head(10))
display(df.iloc[0][1])
display(df.iloc[1][1])

,orig_skills,modif_skills,skills_count
0,Python\tSQL\tРабота в команде\tOpenCV\tML,Python_SQL_Работа в команде_OpenCV_ML,5
1,Oracle Pl/SQL\tБазы данных\tMachine Learning\t...,Oracle Pl/SQL_Базы данных_Machine Learning_Маш...,9
2,Python\tOpenCV\tTensorflow\tComputer Vision\tН...,Python_OpenCV_Tensorflow_Computer Vision_Нейро...,8
3,Python\tFlask\tLinux\tAWS\tGit,Python_Flask_Linux_AWS_Git,5
4,Python\tSQL\tNLP\tScikit-learn\tMachine Learning,Python_SQL_NLP_Scikit-learn_Machine Learning,5
5,Python\tSQL\tNLP\tScikit-learn\tMachine Learning,Python_SQL_NLP_Scikit-learn_Machine Learning,5
6,Python\tSQL\tNLP\tScikit-learn\tMachine Learning,Python_SQL_NLP_Scikit-learn_Machine Learning,5
7,Python\tSQL\tNLP\tScikit-learn\tMachine Learning,Python_SQL_NLP_Scikit-learn_Machine Learning,5
8,Python\tFlask\tLinux\tAWS\tGit,Python_Flask_Linux_AWS_Git,5
9,Python\tПоиск информации в интернет\tC++\tМате...,Python_Поиск информации в интернет_C++_Математ...,5


'Python_SQL_Работа в команде_OpenCV_ML'

'Oracle Pl/SQL_Базы данных_Machine Learning_Машинное обучение_Git_Data science_Python_Работа с базами данных_Математические модели'

In [640]:
# запрос для подсчета среднего кол-ва навыков
query_6_5 = f'''
SELECT 
	round(avg(length(key_skills) - length(REPLACE(key_skills, CHR(9), '')) +1), 2) as avg_count
FROM public.vacancies
where (lower(name) like '%data scientist%' OR 
	   lower(name) like '%data science%' OR 
	   lower(name) like '%исследователь данных%' OR 
	   (name like '%ML%' and name NOT like '%HTML%' and NOT name like '%QML%') OR --дополнительно исключаем лишние вакансии
	   lower(name) like '%machine learning%' OR 
	   lower(name) like '%машинн%обучен%') AND
	   key_skills IS NOT NULL
'''

# результаты запроса
df = pd.read_sql_query(query_6_5, connection)
df

,avg_count
0,6.43


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [641]:
# запрос для визуального исследования данных (до группировки)
query_6_6 = f'''
SELECT 
	experience, 
	(coalesce(salary_from, 0)+coalesce(salary_to, 0))/((salary_from IS NOT NULL)::int + (salary_to IS NOT NULL)::int) as avg_salary
FROM public.vacancies
where (lower(name) like '%data scientist%' OR 
	   lower(name) like '%data science%' OR 
	   lower(name) like '%исследователь данных%' OR 
	   (name like '%ML%' and name NOT like '%HTML%' and NOT name like '%QML%') OR --дополнительно исключаем лишние вакансии
	   lower(name) like '%machine learning%' OR 
	   lower(name) like '%машинн%обучен%') AND
	   (salary_from IS NOT NULL OR salary_to IS NOT NULL)
'''

# результаты запроса
df = pd.read_sql_query(query_6_6, connection)
display(df.head(10))

,experience,avg_salary
0,От 1 года до 3 лет,60000
1,От 1 года до 3 лет,75000
2,Нет опыта,37000
3,Нет опыта,50000
4,От 3 до 6 лет,119000
5,От 1 года до 3 лет,110000
6,От 3 до 6 лет,233794
7,Нет опыта,72500
8,От 3 до 6 лет,150000
9,От 3 до 6 лет,233794


In [642]:
# запрос для подсчета средней ЗП
query_6_5 = f'''
SELECT 
	experience, 
	round(avg(
		 (coalesce(salary_from, 0)+coalesce(salary_to, 0))/((salary_from IS NOT NULL)::int + (salary_to IS NOT NULL)::int)
	)) as avg_salary
FROM public.vacancies
where (lower(name) like '%data scientist%' OR 
	   lower(name) like '%data science%' OR 
	   lower(name) like '%исследователь данных%' OR 
	   (name like '%ML%' and name NOT like '%HTML%' and NOT name like '%QML%') OR --дополнительно исключаем лишние вакансии
	   lower(name) like '%machine learning%' OR 
	   lower(name) like '%машинн%обучен%') AND
	   (salary_from IS NOT NULL OR salary_to IS NOT NULL)
GROUP BY experience
'''

# результаты запроса
df = pd.read_sql_query(query_6_5, connection)
df

,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,142626.0
2,От 3 до 6 лет,243115.0


***

**Выводы по предметному анализу**

* Найдено 1771 вакансия, где в названии встречается 'data'/'данн'. Это примерно 3,6% от общего числа вакансий. Однако, не все из них имеют отношение к DS.

* Было принято считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
    + 'data scientist'
    + 'data science'
    + 'исследователь данных'
    + 'ML' (здесь не нужно брать вакансии по HTML)
    + 'machine learning'
    + 'машинн%обучен%'  
    
    Этому условию соответсвует около 480 вакансий.

* В ходе исследования данных были обнаружены вакансии, которые ошибочно попали под указанные условия фильтраци: например, "C++ разработчик (QML)". Запрос был дополнен условием "не брать вакансии по QML".

* Так же были замечены вакансии в сфере DS, которые не попадают под указанные критерии. Например, "DataOps", "Data engineer", "Data analyst". Возможно их следует включить в запрос.

* В среднем для вакансий DS указывают 6-7 ключевых навыков. Основные из них: Python, SQL, Machine learning, Git, Pandas, Nupmy и др.

* Чаще требуются специалисты с опытом работы от 1 года и более (95% вакансий DS), но вакансии для начинающих специалистов также присутвуют на рынке.

* От длительности опыта работы зависит уровень зп. Начинающий специалистам в среднем готовы платить около 75 тыс. руб. Через пару лет можно претендовать на зп в 2 раза выше (ок. 140 тыс. руб.). Специалисты с опытом от 3 до 6 лет могут расчитывать в среднем на зп ок. 243 тыс. руб. Скорее всего это связано с тем, что с опытом повышается уровень квалификации специалиста. Вероятно данные показатели могут розниться в зависимости от региона (рекомендуются дополнительные исследования).

# Общий вывод по проекту

Проведен предварительный анализ данных с сайта hh.ru с целью дальнейшего создания модели машинного обучения для рекомендации вакансий соискателям, претендующим на позицию Data Scientist.
Основной инструмент - SQL запросы.

1. Наибольшее количество вакансий размещены в крупных городах
2. В большинстве вакансий требуются сотрудники на полный рабочий день с полной занятостью с опытом работы от 1 года до 3 лет
3. Уровень ЗП чаще всего растет пропорционально опыту работы
4. На позицию Data Scientist в подавляющем большинстве требуются специалисты с опытом работы от 1 года и более (95% вакансий DS), но присутсвуют вакансии и для начинающих специалистов. В качестве основных ключевых навыков указываются: Python, SQL, Machine learning, Git, Pandas, Nupmy и др.

5. Справочники регионов и сфер деятельности содержат множество неповторяющихся значений, что делает эти признаки слабоинформативным для анализа. Рекомендуется в дальнейшем преобразование данных с объединением признаков в группы средствами Pandas.
6. Рекомендуется дополнительные исследования зависимости ЗП от региона и сферы деятельности (после преобразования этих признаков из п.5)
7. Присутствуют потенциально аномальные значения в зарплатной вилке (слишком низкие значения: 30, 70 руб. и т.д.), возможно данные ошибочно загружены не в рублях или не учтена кратность, например тыс.руб.